# Mild introduction to Xarray

Based on Ondřej Grover's [PyData Prague Talk 5.10.2020](https://github.com/smartass101/xarray-pydata-prague-2020)

Motivation for xarray:
* Labels and named dimensions for numpy --> indexing, broadcasting.
* More dimensions than Pandas (essentially) 2-D DataFrame.

### Enter `xarray`

<div>
<img src="https://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" alt="xarray logo" style="float:left; width:49%;">
<img src="https://xarray.pydata.org/en/stable/_images/numfocus_logo.png" alt="NumFOCUS logo" style="float:left; width: 49%;">
</div>


- pure Python with (optional) compiled dependencies (NumPy, Pandas, NetCDF, ...)
- superb, well-structured documentation with many examples: [xarray.pydata.org](https://xarray.pydata.org/en/stable/index.html)



Brief historical context:
- evolution of an internal tool developed at The Climate Corporation 
- released as open source in May 2014 as `xray`, renamed in January 2016
- became a fiscally sponsored project of NumFOCUS in August 2018


### Xarray's answers to the motivation: 

* assigns names to dimensions ... axis=N -> dim='name' (in wrapped functions/methods)
    * no need to remember dimension ordering
    * if dimension missing : dim='name' fails explicitly vs. axis=N might pack a surprise for later ...
* uses and generalizes Pandas indexing machinery into multiple-dimensions
* handles broadcasting according to dimensions names
    * checks if dimension sizes match (and/or aligns by associated coordinate indices if available)
    * reshaping, transposing, ... for arithmetic operations
    * handles *ufunc* broadcasting via `apply_ufunc` helper API
 

## `DataArray`:  Multi-dimensional array with associated coordinates

[`DataArray`](https://xarray.pydata.org/en/stable/data-structures.html#dataarray)
is a multi-dimensional array with extra information added. It contains (to quote the official documentation):

- `values`: a numpy.ndarray holding the array’s values

- `dims`: dimension names for each axis (e.g., ('x', 'y', 'z'))

- `coords`: a dict-like container of arrays (coordinates) that label each point (e.g., 1-dimensional arrays of numbers, datetime objects or strings)

- `attrs`: dict to hold arbitrary metadata (attributes)

- `name`: optional name of the array

The `dims` are always present (if you do not see them, they are created automatically), `coords` are optional.
When specifying axes (e.g. for partial aggregations), you use their "dim" names instead of numbers (difference from `numpy`)
and you can index the points using the coordinate values (this is similar to `pandas` `.loc` indexing).

Let's create an array that contains intensity of a laser beam at each point in a plane, as evolving in time.
(Note: sorry for the Gaussian approximation in all dimensions!):

In [ ]:
import numpy as np
import xarray as xr

# Coordinates
x = np.linspace(-10, 10, 11)
y = np.linspace(-10, 10, 11)
t = np.linspace(-1e-12, 1e-12, 10)

# Profile (in arbitrary units)
# Use bradcasting to form a proper cartesian product
data = (
    np.exp(-((x[:, np.newaxis, np.newaxis] / 5) ** 2))
    * np.exp(-((y[np.newaxis, :, np.newaxis] / 5) ** 2))
    * np.exp(-((t[np.newaxis, np.newaxis, :] / 5e-13) ** 2))
)

laser_intensity = xr.DataArray(
    data, 
    dims=('x', 'y', 't'),                   # Names of the "axes"
    name='intensity',
    coords={'x': x, 'y': y, 't': t},        # Values for the dimensions (optional or partial)
    attrs={'profile_type': 'gaussian'},     # Arbitrary user-defined data
)

laser_intensity

All the specified information is accessible via eponymous properties:

In [ ]:
laser_intensity.values

In [ ]:
laser_intensity.dims

In [ ]:
laser_intensity.coords

Note that the coordinates themselves are `DataArray`s:

In [ ]:
laser_intensity.coords["t"]

The coordinates (unless there is a name clash) can be accessed directly as keys of the array or even as attributes.

In [ ]:
laser_intensity["t"]

## Indexing

Similarly to pandas, you can access individual values (and slices) 
in `DataArrays` (and `Datasets`, see below) using both the numeric order
and labels (from the coordinates).

To bring the similarility even further, xarray automatically creates
pandas indices for you:

In [ ]:
laser_intensity.indexes

To select the first 5 measurements for the top-left point (before t=0),
you can use numpy-like indexing with the correct order of axes:

In [ ]:
laser_intensity[0,0,:5]

Like in pandas, you can use the `.loc` accessor and provide value-based
indices:

In [ ]:
laser_intensity.loc[-10,-10,-np.inf:0]

Even better:

In [ ]:
laser_intensity.loc[
    {"x": -10,
     "y": -10,
     "t": slice(-np.inf, 0)  # We must construct the slice manually :-(
    }
]

In [ ]:
laser_intensity.loc[0,0,0]

The `.sel` method works similarly but it can also find a "reasonable" value whenever there is no exact match in the coordinates:

In [ ]:
laser_intensity.sel(x=0, y=0, t=0, method="nearest")

Alternatively, you may want to interpolate (if you know what you are doing) to a point or array of points:

In [ ]:
laser_intensity.interp(x=0, y=0, t=np.linspace(0, 1e-12, 10), method="linear")  # The peak around 0 is symmetric :-/

In [ ]:
laser_intensity.interp(x=0, y=0, t=np.linspace(0, 1e-12, 10), method="cubic")

## Arithmetics & aggregations

With respect to traditional maths operators, the arrays work intuitively like you would expect
from numpy or pandas, matching on dimension names/values and broadcasting when a dimension is not present.


In [ ]:
laser_intensity / 2

In [ ]:
laser_intensity > 1e-6

In [ ]:
# The mean intensity for each time:
mean_intensity = laser_intensity.mean(dim=("x", "y"))
mean_intensity

As an example, let's add the information about laser power that depends
only on time (and is shifted because of non-zero optical distance) and is 
also different at different wavelengths:

In [ ]:
λ = [343, 515, 1030, 2060]   # Note that greek letters are valid identifiers in Python 3!
relative = np.array([0.01, 0.5, 1, 0.3])
t_λ = np.linspace(-1e-12, 3e-12, 19)[5:]   # We have data for different times (but some of them match

power = xr.DataArray(
    1e12 * np.exp(-((t_λ[:,np.newaxis] - 8e-13) / 5e-13)**2) * relative[np.newaxis,:],
    dims=("t", "λ"),
    coords={"t": t_λ, "λ": λ}
)
power

And let's do some maths with the two arrays. Remember, that they do not share all coordinates (λ) and that in one
coordinate (t), there are values missing in both arrays. The result automatically expands the missing dimension
and takes into account only the intersection of shared coordinates.

In [ ]:
mean_intensity / power  # Sorry for the nonsensical physics interpretation

## `Dataset`: Container of DataArrays

[`Dataset`](https://xarray.pydata.org/en/stable/data-structures.html#dataset) is to `DataArray` roughly what pandas' `DataFrame` is to `Series`. It is collection of *named* DataArrays that
may share zero to all coordinates => the resulting `.coords` is **union** of individual `.coords`.

Note that unlike DataFrame/Series in pandas, the arrays in a dataset do not have 
to have the same dimensions.

We can combine the power with intensity map in a common dataset:

In [ ]:
laser = xr.Dataset(
    {
        "power": power,
        "intensity": laser_intensity
    }
)
laser

For simplicity, we created the dataset from two previously created
`DataArray` but you may create it from numpy arrays directly,
providing the dims and/or coordinates.

The `dims` property is (unlike in the `DataArray`) is not a tuple
of names but a dictionary of names -> size, giving the idea about
the shape.

In [ ]:
laser.dims

The `data_vars` property contains all arrays that make up the dataset.

In [ ]:
laser.data_vars

### Updating datasets

In [ ]:
laser.assign(total_power=laser["power"].sum("λ"))


In [ ]:
laser["mean_intensity"] = mean_intensity
laser

In [ ]:
del laser["mean_intensity"]
laser

## Plotting with matplotlib using the `.plot` accessor
- Auto-magic `.plot()` call selects method based on dimensionality
- axis labels from `description` and `units` attributes (if available)

In [ ]:
power.sum(dim="λ").plot();   # more than 2D -> histogram

In [ ]:
laser_intensity.loc[{"t": -1e-12}].plot();

In [ ]:
laser_intensity.interp(t=-1e-12, x=np.linspace(-10, 10, 200), y=np.linspace(-10, 10, 200)).plot(cmap="Reds");

In [ ]:
power.plot.line(col='λ')   # 3D array faceted to 2D

## Many other things not covered (but similar to Pandas or domain specific)

Go check out the docs at [xarray.pydata.org](https://xarray.pydata.org/en/stable/index.html)
- stacking (flattening) dimensions into `MultiIndex`
- group-by (split-apply-combine) operations
- rolling windows, coarsening, weighted aggregations
- combining (`xr.concat`) data
- interpolating data
- polynomial fitting
- squeezing, expanding, reordering dimensions
- IO backends
- registering accessors (composition over inheritance)
- visualization plugins
- vectorized functions
- parallelization
- ...

### Further reading

- [Xarray official documentation](http://xarray.pydata.org/en/stable/)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=77a5caea-ff40-471d-8b4b-98dc66dd30c3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>